<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C_%EA%B0%9C%EC%A0%952%ED%8C%90_ch9_7~9_p_625~647.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

## SGD 기반 잠재 요인 협업 필터링

- 목표: 행렬 분해(Matrix Factorization)를 이용한 영화 추천 시스템 구현
- 방법: 확률적 경사 하강법(SGD)을 활용한 행렬 분해
- 배경:
  - SVD는 결측치에 민감 → 사용자-아이템 평점 행렬엔 널(null)이 많아 적합하지 않음
  - SGD나 ALS 기반 행렬 분해가 일반적으로 사용됨
- 활용 코드:
  - `get_rmse()` 함수: RMSE 계산용
  - `matrix_factorization()` 함수 정의:
    - 입력:  
      - `R`: 사용자-아이템 평점 행렬  
      - `K`: 잠재 요인의 수  
      - `steps`: 반복 횟수 (기본 200)  
      - `learning_rate`: 학습률 (기본 0.01)  
      - `r_lambda`: L2 규제 계수 (기본 0.01)
- 활용 예제: 사용자에게 개인화된 영화 추천 제공

✅ get_rmse() 함수 설명

목적: 실제 평점 행렬 R과 예측 평점 행렬(P×Qᵀ) 사이의 RMSE를 계산

입력값:

- R: 원본 사용자-아이템 평점 행렬

- P: 사용자 × 잠재요인 행렬

- Q: 아이템 × 잠재요인 행렬

- non_zeros: 실제 평점이 존재하는 위치 인덱스 리스트 (예: [(0,0), (0,1), ...])

절차:

1. P와 Q.T의 내적으로 예측 평점 행렬 생성

2. 실제 평점이 있는 위치의 값만 추출

3. 해당 위치의 실제 평점과 예측값으로 MSE 계산 → RMSE 반환

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

📌 matrix_factorization() 함수 설명

✅ 목적

SGD(확률적 경사 하강법)를 이용해 사용자-아이템 평점 행렬을 두 개의 잠재 요인 행렬(P, Q)로 분해하여 학습


⚙️ 절차

1. 사용자 행렬 `P`와 아이템 행렬 `Q`를 정규분포로 무작위 초기화
2. 실제 평점이 존재하는 위치만 추출하여 `non_zeros` 리스트 생성
3. 각 스텝(step)마다 다음을 반복:
   - 각 평점 위치에 대해 예측값과 실제값의 차이(`eij`) 계산
   - L2 정규화를 반영해 `P`와 `Q`를 SGD 방식으로 업데이트
   - 10 step마다 RMSE 출력
4. 최종적으로 학습된 `P`, `Q` 행렬 반환

📤 출력값

- `P`: 사용자 잠재 요인 행렬 (shape: 사용자 수 × K)
- `Q`: 아이템 잠재 요인 행렬 (shape: 아이템 수 × K)

In [2]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)

    return P, Q

- 영화 평점 행렬 데이터를 새롭게 DataFrame으로 로딩한 뒤에 다시 사용자-아이템 평점 행렬로 만들기

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/25-1 ESAA OB/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/25-1 ESAA OB/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

In [6]:
# columns='title'로 title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

- matrix_factorization() 함수를 사용해 평점 행렬을 분해
- K=50, steps=200, 학습률과 정규화 계수는 0.01로 설정

In [7]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01,
                            r_lambda=0.01)
pred_matrix = np.dot(P, Q.T) # 원래 평점 행렬의 빈 부분까지 예측값으로 채워진 형태

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.2068545530233154
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720406
### iteration step :  100  rmse :  0.17742927527209104
### iteration step :  110  rmse :  0.1716522696470749
### iteration step :  120  rmse :  0.16695181946871726
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679646
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715425
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  rm

- 반환된 예측 사용자一아이템 평점 행렬을 영화 타이틀을 칼럼명으로 가지는 DataFrame으로 변경

In [8]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


- 사용자 9번이 보지 않은 영화 목록을 get_unseen_movies()로 추출

- 잠재 요인 기반 예측 평점 행렬을 이용해 recomm_movie_by_userid()로 상위 10개 영화 추천

- 추천 결과를 DataFrame으로 정리하여 예측 평점(pred_score) 확인

In [9]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함.
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]

    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()

    # 모든 영화명을 list 객체로 만듬.
    movies_list = ratings_matrix.columns.tolist()

    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [10]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [11]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies =recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies =pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                            columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


* 잠재 요인 협업 필터링은 아이템 기반 협업 필터링과 다른 추천 결과를 보임
* ‘이창’, ‘사우스파크’, ‘라운더스’, ‘블레이드 러너’, ‘가타카’ 등 다소 무거운 주제의 영화들이 추천됨

# 파이썬 추천 시스템 패키지 一 Surprise

* 추천 시스템 전용 파이썬 패키지인 Surprise를 소개
* 사이킷런처럼 사용하기 쉬운 API를 제공하며, pip 또는 conda로 설치 가능
* 기존 구현보다 최적화된 추천 시스템 개발에 유용함

In [ ]:
import surprise

print(surprise.__version__)

1.1.1


## Surprise 를 이용한 추천 시스템 구축

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

- Dataset 클래스를 사용하여 데이터를 로딩
- MovieLens 데이터셋을 `load_builtin()`으로 로딩
- `train_test_split()`을 사용해 데이터를 학습용과 테스트용으로 분리

In [ ]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

- SVD 알고리즘 객체 생성: `algo = SVD()`
- `fit()`을 사용해 학습 데이터 세트로 추천 알고리즘 학습


In [ ]:
algo = SVD(random_state=0)
algo.fit(trainset)

- test(): 테스트 데이터 세트 전체에 대해 추천 평점 예측
- 예제: 테스트 데이터 세트에서 추천 평점 생성 후, 첫 5개 항목 추출


In [ ]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

- test() 메서드는 25,000개의 `Prediction` 객체 리스트를 반환
- `Prediction` 객체는 사용자 아이디(uid), 영화 아이디(iid), 실제 평점(r_ui), 예측 평점(est)을 포함
- `was_impossible` 속성은 예측 불가능한 데이터에 대해 True로 설정됨 (여기선 모두 False)
- `Prediction` 객체 속성 접근: `객체명.uid`, `객체명.iid`, `객체명.est`와 같은 형식으로 사용


In [ ]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

- predict() 메서드는 개별 사용자와 아이템에 대한 추천 평점을 예측
- 사용자 아이디와 아이템 아이디를 인자로 입력하면, 추천 예측 평점을 포함한 정보를 반환
- 기존 평점 정보(r_ui)는 선택 사항이며, 사용자 아이디와 아이템 아이디는 문자열로 입력


In [ ]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


- predict()는 개별 사용자와 아이템에 대해 추천 예측 평점(est)을 반환
- test()는 모든 사용자와 아이템에 대해 predict()를 반복하여 실행한 결과
- 테스트 데이터 세트로 추천 예측 평점과 실제 평점의 차이를 평가
- accuracy 모듈의 rmse()를 사용하여 RMSE로 성능 평가


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

## Surprise 주요 모듈 소개

- **Dataset.load_builtin(name='ml-100k')**
  - 무비렌즈 아카이브 FTP 서버에서 데이터를 내려받습니다
  - `ml-100k` 또는 `ml-1M` 데이터를 선택하여 로딩
  
- **Dataset.load_from_file(file_path, reader)**
  - OS 파일에서 데이터를 로딩. 콤마, 탭 등으로 구분된 포맷을 처리
  
- **Dataset.load_from_df(df, reader)**
  - 판다스 DataFrame에서 데이터를 로딩.
  - 데이터는 반드시 `사용자 아이디`, `아이템 아이디`, `평점` 순으로 칼럼이 정렬되어야 함

### OS 파일 데이터를 Surprise 데이터 세트로 로딩

- **Dataset.load_from_file** API를 사용해 사용자-아이템 평점 데이터를 로딩
- **주의사항**: 데이터 파일에 칼럼명 헤더가 포함되지 않도록 해야 함
- 예시:
  - `ratings.csv` 파일은 헤더를 포함하므로, `pandas`의 `to_csv()`를 이용해 헤더를 제거한 후 `ratings_noh.csv`로 저장


In [ ]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)


###ratings_noh.csv 파일 로딩

- **Reader 클래스**: 데이터 파일의 파싱 포맷을 정의
  - 칼럼: 사용자 아이디, 아이템 아이디, 평점, 타임스탬프
  - 분리문자: 콤마, 평점 단위: 0.5, 최대 평점: 5
- **Dataset.load_from_file()**: `ratings_noh.csv` 파일에서 사용자-아이템 평점 데이터를 로딩하며, `timestamp` 칼럼은 제외


In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

- line_format: 칼럼 순서를 공백으로 구분해 문자열로 입력
- sep: 칼럼을 구분하는 문자 (디폴트: 콤마)
- rating_scale: 평점의 최소-최대 범위 (디폴트: (1, 5), 예시: (0.5, 5))

- n_factors: 잠재 요인 크기 K 값을 50으로 설정
- 학습 후 테스트 데이터 세트에서 예측 평점 계산
- 예측 평점과 실제 평점의 차이를 RMSE로 평가

In [ ]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

- Dataset.load_from_df() 사용하여 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
- 주의: DataFrame은 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 따라야 함
- 예시 코드: `Dataset.load_from_df(ratings[['userid', 'movield', 'rating']], reader)`
- 이후 SVD 추천 예측 적용

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

Surprise에서 자주 사용되는 추천 알고리즘 클래스:

- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링
- KNNBasic: 최근접 이웃 협업 필터링.
- BaselineOnly: 사용자 및 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
- 추가로 SVD++, NMF 등 다양한 알고리즘을 지원

#### SVD 알고리즘
SVD 알고리즘의 비용 함수는 사용자 베이스라인 편향을 고려하며 Regularization을 적용하여 예측 평점을 구함.

주요 파라미터:

- n_factors: 잠재 요인 수
- n_epochs: 반복 횟수
- biased: 베이스라인 사용자 편향 적용 여부

#### 벤치마크 결과

- SVD++: 가장 낮은 RMSE와 MAE를 기록했으나 시간이 오래 걸림
- SVD와 k-NN Baseline: 상대적으로 좋은 성능을 보였고, k-NN Baseline은 Baseline을 결합한 경우 성능이 크게 향상됨


### 베이스라인 평점

베이스라인 평점은 개인의 평점 성향을 반영하여 아이템에 대한 편향성을 고려한 평점 계산 방식

일반적으로 베이스라인 평점은 다음의 공식을 사용하여 계산:

- **전체 평균 평점**: 모든 사용자가 평가한 아이템의 평균 평점
- **사용자 편향 점수**: 각 사용자가 평가한 아이템의 평균 평점에서 전체 평균 평점을 뺀 값
- **아이템 편향 점수**: 각 아이템에 대한 평균 평점에서 전체 평균 평점을 뺀 값

### 예시:
1. 전체 평균 평점: 3.5
2. 사용자 A의 평균 평점: 3.0 → 사용자 편향 점수: 3.0 - 3.5 = -0.5
3. '어벤저스 3편'의 평균 평점: 4.2 → 아이템 편향 점수: 4.2 - 3.5 = 0.7

따라서 사용자 A의 '어벤저스 3편'에 대한 베이스라인 평점은: 3.5 - 0.5 + 0.7 = 3.7


### 교차 검증과 하이퍼 파라미터 튜닝

- Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 `cross_validate()`와 `GridSearchCV` 클래스를 제공

- `cross_validate()` 함수는 `surprise.model_selection` 모듈에 포함되어 있으며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 설정하여 교차 검증을 수행

- 다음 예제에서는 `cross_validate()`를 사용하여 `ratings.csv` 파일을 DataFrame으로 로딩한 데이터를 5개의 학습/검증 폴드로 나누고, RMSE와 MAE를 성능 평가 지표로 사용하여 교차 검증을 수행

- `cross_validate()` 함수의 주요 인자:
  - 알고리즘 객체
  - 데이터
  - 성능 평가 방법 (`measures`)
  - 폴드 데이터 세트 개수 (`cv`)


In [ ]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8738  0.8725  0.8702  0.8726  0.8850  0.8748  0.0052  
MAE (testset)     0.6724  0.6704  0.6712  0.6688  0.6795  0.6725  0.0037  
Fit time          12.18   12.06   12.06   12.26   11.64   12.04   0.21    
Test time         0.27    0.28    0.43    0.26    0.24    0.30    0.07    


{'test_rmse': array([0.87379402, 0.87245487, 0.87022605, 0.87255353, 0.88504587]),
 'test_mae': array([0.67242816, 0.67037854, 0.67123931, 0.66883741, 0.67948228]),
 'fit_time': (12.17864179611206,
  12.05741286277771,
  12.057470083236694,
  12.255924940109253,
  11.644625186920166),
 'test_time': (0.27306103706359863,
  0.28406333923339844,
  0.4320967197418213,
  0.2600581645965576,
  0.24205303192138672)}

- `cross_validate()`:
  - 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 출력
  - 교차 검증 세트의 분할 방식에 따라 출력 결과가 달라질 수 있음

- `GridSearchCV`:
  - 사이킷런의 `GridSearchCV`와 유사하게 하이퍼 파라미터 최적화 수행
  - SVD의 경우 `n_epochs` (반복 횟수)와 `n_factors` (잠재 요인 크기)를 튜닝
  - 예시: `n_epochs: [20, 40, 60]`, `n_factors: [50, 100, 200]`로 변경하여 최적 하이퍼 파라미터 도출
  - 데이터: `ratings.csv` 사용


In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769381880851433
{'n_epochs': 20, 'n_factors': 50}


### Surprise 를 이용한 개인화 영화 추천 시스템 구축

- Surprise를 이용해 개인화된 영화 추천 구현
- 기존 예제는 `fit()`으로 학습하고 `test()`로 예측, MSE/RMSE로 성능 평가
- 새로운 예제에서는 학습 데이터와 테스트 데이터 분리 없이 전체 데이터를 학습에 사용
- Surprise에서 데이터 세트를 `train_test_split()`을 통해 `TrainSet` 객체로 변환해야 `fit()`을 사용할 수 있음


In [ ]:
# 다음 코드는 train_test_split( )으로 분리되지 않는 데이터 세트에 fit( )을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

- 데이터 세트 전체를 학습 데이터로 사용하려면 `DatasetAutoFolds` 클래스를 이용
- `DatasetAutoFolds` 객체 생성 후 `build_full_trainset()` 메서드를 호출하면 전체 데이터를 학습 데이터 세트로 사용 가능


In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()

- `DatasetAutoFolds`의 `build_full_trainset()` 메서드를 이용해 전체 데이터를 학습 데이터로 생성
- SVD 알고리즘을 사용해 학습 수행
- 학습 후, 특정 사용자가 아직 보지 않은 영화 목록을 확인하여 추천 진행


In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- 특정 사용자(userid = 9)를 지정하여 예측 수행
- 영화 ID 42번에 대한 평점을 예측
- 영화의 상세 정보는 `movies.csv` 파일에서 로딩하여 DataFrame으로 사용


In [ ]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


- `predict()` 메서드를 사용하여 영화 ID 42에 대한 사용자 ID 9의 예상 평점 계산
- `predict()` 메서드 내 `userid`와 `movielid`는 문자열 값으로 입력해야 함


In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


- 추천 예측 평점은 `est` 값으로 확인, 예: 3.13
- 사용자가 평점을 매기지 않은 전체 영화를 추출하고 예측 평점 순으로 추천
- 기존 `get_unseen_movies()` 대신 `get_unseen_surprise()` 함수 사용
- 이 함수는 사용자 ID 9가 아직 평점을 매기지 않은 영화 정보를 반환


In [ ]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


- 사용자 9번은 9742개 영화 중 46개만 평점 매김
- 추천 대상 영화는 9696개
- `recomm_movie_by_surprise()` 함수 생성:
  - 인자: 학습된 추천 알고리즘 객체, 사용자 아이디, 추천 대상 영화 리스트, 추천 상위 N개 개수
  - `predict()` 메서드 호출 후 Prediction 객체 리스트로 저장
  - 리스트를 예측 평점 순으로 정렬 후, Top-N개의 영화 정보 반환 (영화 아이디, 제목, 예측 평점)


In [ ]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est

    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


- 9번 사용자에게 추천된 영화:
  - '유주얼 서스펙트', '펄프픽션', '양들의 침묵', '대부', '좋은 친구들' (서스펜스/스릴러/범죄)
  - '스타워즈' (액션 영화)
- Surprise 패키지는 복잡한 알고리즘 구현 없이 간결한 API로 추천 시스템 구축 가능
